In [1]:
import csv
import time
import signal
import numpy as np
import pandas as pd
import multiprocessing as mp
from multiprocessing import Process
from multiprocessing import Semaphore
import statsmodels.formula.api as smf

In [109]:
data = pd.read_csv(
    '/Users/ljubomir/Documents/neurohackademy_2021/project/NBR/data/frontal2D.csv',
    sep=",",
    header=0
).apply(pd.to_numeric, errors='ignore')
data.columns = [c.replace('.', '_') for c in data.columns]
data.head()

,Group,Sex,Age,FAG_FAD,FAG_F1G,FAD_F1G,FAG_F1D,FAD_F1D,F1G_F1D,FAG_F1OG,...,ORD_GRD,SMAG_GRD,SMAD_GRD,COBG_GRD,COBD_GRD,FMG_GRD,FMD_GRD,FMOG_GRD,FMOD_GRD,GRG_GRD
0,Control,F,8.52,0.353834,-0.079097,-0.142089,-0.098187,0.182184,0.136119,-0.136528,...,-0.227927,0.183839,0.024063,0.375259,0.291237,0.334427,0.422445,0.523587,0.556687,0.819152
1,Control,M,16.16,1.063398,-0.007951,-0.221031,-0.415336,-0.500766,0.711215,-0.166893,...,-0.118639,0.207211,0.211242,0.350441,0.297225,0.014917,0.089388,0.122101,0.084486,0.765234
2,Patient,M,17.75,1.268696,0.308791,0.250510,-0.044810,-0.015975,0.250217,-0.376892,...,0.000284,-0.257270,-0.211604,0.405767,0.455390,0.197229,0.422473,0.871838,0.769304,1.475135
3,Control,M,12.27,0.483665,0.056052,-0.199248,-0.114940,-0.006603,0.196900,-0.103025,...,-0.207249,-0.137101,-0.137722,0.214525,0.227899,0.390530,0.494048,0.469539,0.183129,0.874235
4,Control,F,12.07,0.735635,0.168888,0.069894,-0.466187,-0.212627,0.144734,-0.400744,...,-0.213785,-0.324647,-0.309591,0.434704,0.326873,0.011111,0.253135,0.660290,0.605073,0.835976


#### Check dataset

In [110]:
data.shape

(48, 381)

In [111]:
brain_positions = pd.unique(np.hstack([d.split('_') for d in data.columns[3:]]))
brain_positions.shape, brain_positions

((28,),
 array(['FAG', 'FAD', 'F1G', 'F1D', 'F1OG', 'F1OD', 'F2G', 'F2D', 'F2OG',
        'F2OD', 'F3OPG', 'F3OPD', 'F3TG', 'F3TD', 'F3OG', 'F3OD', 'ORG',
        'ORD', 'SMAG', 'SMAD', 'COBG', 'COBD', 'FMG', 'FMD', 'FMOG',
        'FMOD', 'GRG', 'GRD'], dtype='<U5'))

In [108]:
bp_inter = []

for ida, _ in enumerate(brain_positions):
    for idb, _ in enumerate(brain_positions[ida+1:]):
        bp_inter.append('%s_%s' % (brain_positions[ida], brain_positions[idb]))
len(bp_inter)

1431

#### Define input constants

In [95]:
n_nodes=28
mod='Group + Sex * Age'
alternative="less"
n_perm=5
diag=False
cores=5
thr_t=0.4
thr_p=0.05

In [96]:
def fit_linear_model(data, dependent, formula):
    """

    :param data:
    :param dependent:
    :param formula:
    :return:
    """
    # create a linear model
    # FIXME: we need to figure out a better way to parse formula
    lm = smf.ols(
        formula='%s ~ %s' % (dependent, formula),
        data=data
    )
    res = lm.fit()

    # return t-values and p-values for each independent variable
    res_out = pd.concat([res.tvalues, res.pvalues], axis=1).iloc[1:]
    res_out.columns = ['tvals_%s' % dependent, 'pvals_%s' % dependent]
    return res_out

In [97]:
triu = np.triu(np.ones(5), k=0)
triu_ind = np.where(triu == 0)
triu, triu_ind

(array([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]]),
 (array([1, 2, 2, 3, 3, 3, 4, 4, 4, 4]),
  array([0, 0, 1, 0, 1, 2, 0, 1, 2, 3])))

In [98]:
connection_indices = np.vstack((np.where(np.triu(np.ones(n_nodes), k=0) == 0))).T
connection_indices[:, [0,1]] = connection_indices[:, [1,0]]
connection_indices[:10]

array([[0, 1],
       [0, 2],
       [1, 2],
       [0, 3],
       [1, 3],
       [2, 3],
       [0, 4],
       [1, 4],
       [2, 4],
       [3, 4]])

In [122]:
st = time.time()
lms = []
for column in data.columns[3:]:
    lms.append(
        fit_linear_model(
            data,
            column,
            mod
        )
    )
lms = pd.concat(lms, axis=1)
et = time.time()
print('[perfr-measurment] Fitting models without parallelization %.2f [sec]' % (et - st))

[perfr-measurment] Fitting models without parallelization 1.22 [sec]


In [123]:
lms.head()

,tvals_FAG_FAD,pvals_FAG_FAD,tvals_FAG_F1G,pvals_FAG_F1G,tvals_FAD_F1G,pvals_FAD_F1G,tvals_FAG_F1D,pvals_FAG_F1D,tvals_FAD_F1D,pvals_FAD_F1D,...,tvals_FMG_GRD,pvals_FMG_GRD,tvals_FMD_GRD,pvals_FMD_GRD,tvals_FMOG_GRD,pvals_FMOG_GRD,tvals_FMOD_GRD,pvals_FMOD_GRD,tvals_GRG_GRD,pvals_GRG_GRD
Group[T.Patient],1.252376,0.217202,0.269196,0.789066,1.003822,0.321080,2.096530,0.041955,2.581793,0.013320,...,-2.014865,0.050199,-1.873484,0.067808,-0.428692,0.670287,-0.742091,0.462067,-0.014585,0.988430
Sex[T.M],0.520570,0.605337,1.265877,0.212373,0.389641,0.698726,1.449636,0.154415,-0.029146,0.976883,...,-0.179363,0.858495,-0.332176,0.741370,-0.513080,0.610521,-0.601811,0.550460,-0.149395,0.881941
Age,0.933738,0.355653,-0.108880,0.913804,-0.425130,0.672862,-0.318207,0.751868,-1.624216,0.111639,...,-0.251250,0.802818,-0.035086,0.972174,0.502234,0.618065,-0.397677,0.692836,-0.259062,0.796824
Sex[T.M]:Age,-0.522405,0.604070,-1.052625,0.298392,-0.469325,0.641208,-1.303951,0.199188,0.082814,0.934384,...,0.283109,0.778452,0.337900,0.737083,0.240402,0.811161,0.669974,0.506457,0.092156,0.927002


#### If t-vals selected

In [12]:
df_tvals = lms[[col for col in lms.columns if 'tvals_' in col]]
df_tvals.head()

,tvals_FAG_FAD,tvals_FAG_F1G,tvals_FAD_F1G,tvals_FAG_F1D,tvals_FAD_F1D,tvals_F1G_F1D,tvals_FAG_F1OG,tvals_FAD_F1OG,tvals_F1G_F1OG,tvals_F1D_F1OG,...,tvals_ORD_GRD,tvals_SMAG_GRD,tvals_SMAD_GRD,tvals_COBG_GRD,tvals_COBD_GRD,tvals_FMG_GRD,tvals_FMD_GRD,tvals_FMOG_GRD,tvals_FMOD_GRD,tvals_GRG_GRD
Group[T.Patient],1.252376,0.269196,1.003822,2.096530,2.581793,1.392424,-1.590964,-0.381642,-1.528961,-0.877984,...,1.159379,-1.078823,-0.649621,-0.376400,0.485534,-2.014865,-1.873484,-0.428692,-0.742091,-0.014585
Sex[T.M],0.520570,1.265877,0.389641,1.449636,-0.029146,-0.346909,-0.436225,0.228362,-1.338885,-1.470305,...,1.543478,-2.136063,-1.356843,0.030093,0.426709,-0.179363,-0.332176,-0.513080,-0.601811,-0.149395
Age,0.933738,-0.108880,-0.425130,-0.318207,-1.624216,-1.540859,0.909137,0.503687,-0.576900,-0.094179,...,-0.659243,-1.691688,-0.709231,-0.084929,-0.102182,-0.251250,-0.035086,0.502234,-0.397677,-0.259062
Sex[T.M]:Age,-0.522405,-1.052625,-0.469325,-1.303951,0.082814,0.408965,0.443339,-0.212204,1.115486,1.233601,...,-1.067403,2.194067,1.402942,-0.062722,-0.417274,0.283109,0.337900,0.240402,0.669974,0.092156


In [13]:
strength = (df_tvals.abs() - abs(thr_t))*(1-2*(df_tvals < 0))
strength.head()

,tvals_FAG_FAD,tvals_FAG_F1G,tvals_FAD_F1G,tvals_FAG_F1D,tvals_FAD_F1D,tvals_F1G_F1D,tvals_FAG_F1OG,tvals_FAD_F1OG,tvals_F1G_F1OG,tvals_F1D_F1OG,...,tvals_ORD_GRD,tvals_SMAG_GRD,tvals_SMAD_GRD,tvals_COBG_GRD,tvals_COBD_GRD,tvals_FMG_GRD,tvals_FMD_GRD,tvals_FMOG_GRD,tvals_FMOD_GRD,tvals_GRG_GRD
Group[T.Patient],0.852376,-0.130804,0.603822,1.696530,2.181793,0.992424,-1.190964,0.018358,-1.128961,-0.477984,...,0.759379,-0.678823,-0.249621,0.023600,0.085534,-1.614865,-1.473484,-0.028692,-0.342091,0.385415
Sex[T.M],0.120570,0.865877,-0.010359,1.049636,0.370854,0.053091,-0.036225,-0.171638,-0.938885,-1.070305,...,1.143478,-1.736063,-0.956843,-0.369907,0.026709,0.220637,0.067824,-0.113080,-0.201811,0.250605
Age,0.533738,0.291120,-0.025130,0.081793,-1.224216,-1.140859,0.509137,0.103687,-0.176900,0.305821,...,-0.259243,-1.291688,-0.309231,0.315071,0.297818,0.148750,0.364914,0.102234,0.002323,0.140938
Sex[T.M]:Age,-0.122405,-0.652625,-0.069325,-0.903951,-0.317186,0.008965,0.043339,0.187796,0.715486,0.833601,...,-0.667403,1.794067,1.002942,0.337278,-0.017274,-0.116891,-0.062100,-0.159598,0.269974,-0.307844


In [14]:
if alternative == "two.sided":
    edges = df_tvals.abs() > thr_t
elif alternative == "less":
    edges = df_tvals < thr_t
else:
    edges = df_tvals > thr_t
edges

,tvals_FAG_FAD,tvals_FAG_F1G,tvals_FAD_F1G,tvals_FAG_F1D,tvals_FAD_F1D,tvals_F1G_F1D,tvals_FAG_F1OG,tvals_FAD_F1OG,tvals_F1G_F1OG,tvals_F1D_F1OG,...,tvals_ORD_GRD,tvals_SMAG_GRD,tvals_SMAD_GRD,tvals_COBG_GRD,tvals_COBD_GRD,tvals_FMG_GRD,tvals_FMD_GRD,tvals_FMOG_GRD,tvals_FMOD_GRD,tvals_GRG_GRD
Group[T.Patient],False,True,False,False,False,False,True,True,True,True,...,False,True,True,True,False,True,True,True,True,True
Sex[T.M],False,False,True,False,True,True,True,True,True,True,...,False,True,True,True,False,True,True,True,True,True
Age,False,True,True,True,True,True,False,False,True,True,...,True,True,True,True,True,True,True,False,True,True
Sex[T.M]:Age,True,True,True,True,True,False,False,True,False,False,...,True,False,False,True,True,True,True,True,False,True


In [15]:
edges.sum(axis=1)

Group[T.Patient]    240
Sex[T.M]            283
Age                 284
Sex[T.M]:Age        207
dtype: int64

In [16]:
observations = {}
for var in edges.index:
    e = np.where(edges.loc[var] == 1)
    ci_row = connection_indices[e, 0]
    ci_col = connection_indices[e, 1]
    s = strength.loc[var].values[e]
    observations[var] = pd.DataFrame(np.vstack((e,ci_row,ci_col,s)).T, columns=["2Dcol","3Drow","3Dcol","strn"])

In [17]:
observations['Group[T.Patient]']

,2Dcol,3Drow,3Dcol,strn
0,1.0,0.0,2.0,-0.130804
1,6.0,0.0,4.0,-1.190964
2,7.0,1.0,4.0,0.018358
3,8.0,2.0,4.0,-1.128961
4,9.0,3.0,4.0,-0.477984
...,...,...,...,...
235,373.0,22.0,27.0,-1.614865
236,374.0,23.0,27.0,-1.473484
237,375.0,24.0,27.0,-0.028692
238,376.0,25.0,27.0,-0.342091


#### If p-vals selected

In [18]:
df_pvals = lms[[col for col in lms.columns if 'pvals_' in col]]
df_pvals.head()

,pvals_FAG_FAD,pvals_FAG_F1G,pvals_FAD_F1G,pvals_FAG_F1D,pvals_FAD_F1D,pvals_F1G_F1D,pvals_FAG_F1OG,pvals_FAD_F1OG,pvals_F1G_F1OG,pvals_F1D_F1OG,...,pvals_ORD_GRD,pvals_SMAG_GRD,pvals_SMAD_GRD,pvals_COBG_GRD,pvals_COBD_GRD,pvals_FMG_GRD,pvals_FMD_GRD,pvals_FMOG_GRD,pvals_FMOD_GRD,pvals_GRG_GRD
Group[T.Patient],0.217202,0.789066,0.321080,0.041955,0.013320,0.170953,0.118944,0.704606,0.133598,0.384832,...,0.252699,0.286681,0.519393,0.708470,0.629762,0.050199,0.067808,0.670287,0.462067,0.988430
Sex[T.M],0.605337,0.212373,0.698726,0.154415,0.976883,0.730352,0.664856,0.820447,0.187644,0.148759,...,0.130043,0.038408,0.181912,0.976132,0.671720,0.858495,0.741370,0.610521,0.550460,0.881941
Age,0.355653,0.913804,0.672862,0.751868,0.111639,0.130679,0.368346,0.617052,0.567015,0.925404,...,0.513255,0.097942,0.482009,0.932712,0.919087,0.802818,0.972174,0.618065,0.692836,0.796824
Sex[T.M]:Age,0.604070,0.298392,0.641208,0.199188,0.934384,0.684595,0.659743,0.832951,0.270837,0.224052,...,0.291746,0.033683,0.167814,0.950279,0.678554,0.778452,0.737083,0.811161,0.506457,0.927002


In [19]:
import scipy.stats

In [20]:
qt_2 = scipy.stats.t.ppf(1 - thr_p / 2, data.shape[0] - lms.shape[0] - 1)
qt = scipy.stats.t.ppf(1 - thr_p, data.shape[0] - lms.shape[0] - 1)

In [21]:
if alternative == "two.sided":
    edges = df_pvals < thr_p
    strength = (df_tvals.abs() - qt_2)*(1-2*(df_tvals < 0))
elif alternative == "less":
    edges = pd.DataFrame((df_pvals < 2*thr_p).values & (df_tvals < 0).values, columns=df_pvals.columns, index=df_pvals.index)
    strength = (df_tvals.abs() - qt)*(1-2*(df_tvals < 0))
else:
    edges = pd.DataFrame((df_pvals < 2*thr_p).values & (df_tvals > 0).values, columns=df_pvals.columns, index=df_pvals.index)
    strength = (df_tvals.abs() - qt)*(1-2*(df_tvals < 0))
edges

,pvals_FAG_FAD,pvals_FAG_F1G,pvals_FAD_F1G,pvals_FAG_F1D,pvals_FAD_F1D,pvals_F1G_F1D,pvals_FAG_F1OG,pvals_FAD_F1OG,pvals_F1G_F1OG,pvals_F1D_F1OG,...,pvals_ORD_GRD,pvals_SMAG_GRD,pvals_SMAD_GRD,pvals_COBG_GRD,pvals_COBD_GRD,pvals_FMG_GRD,pvals_FMD_GRD,pvals_FMOG_GRD,pvals_FMOD_GRD,pvals_GRG_GRD
Group[T.Patient],False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
Sex[T.M],False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
Age,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
Sex[T.M]:Age,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
edges.sum(axis=1)

Group[T.Patient]    49
Sex[T.M]            26
Age                 26
Sex[T.M]:Age        10
dtype: int64

In [23]:
strength

,tvals_FAG_FAD,tvals_FAG_F1G,tvals_FAD_F1G,tvals_FAG_F1D,tvals_FAD_F1D,tvals_F1G_F1D,tvals_FAG_F1OG,tvals_FAD_F1OG,tvals_F1G_F1OG,tvals_F1D_F1OG,...,tvals_ORD_GRD,tvals_SMAG_GRD,tvals_SMAD_GRD,tvals_COBG_GRD,tvals_COBD_GRD,tvals_FMG_GRD,tvals_FMD_GRD,tvals_FMOG_GRD,tvals_FMOD_GRD,tvals_GRG_GRD
Group[T.Patient],-0.428695,-1.411875,-0.677249,0.415459,0.900722,-0.288646,0.090106,1.299429,0.152110,0.803086,...,-0.521691,0.602248,1.031450,1.304670,-1.195537,-0.333794,-0.192413,1.252378,0.938979,1.666485
Sex[T.M],-1.160501,-0.415194,-1.291430,-0.231435,1.651924,1.334162,1.244846,-1.452709,0.342185,0.210766,...,-0.137593,-0.454993,0.324228,-1.650978,-1.254362,1.501708,1.348895,1.167990,1.079260,1.531676
Age,-0.747333,1.572191,1.255941,1.362863,0.056855,0.140212,-0.771934,-1.177383,1.104171,1.586892,...,1.021828,-0.010617,0.971839,1.596142,1.578889,1.429821,1.645985,-1.178836,1.283394,1.422009
Sex[T.M]:Age,1.158666,0.628446,1.211745,0.377120,-1.598257,-1.272105,-1.237732,1.468867,-0.565584,-0.447469,...,0.613668,0.512996,-0.278129,1.618349,1.263797,-1.397962,-1.343171,-1.440669,-1.011097,-1.588915


In [24]:
observations = {}
for var in edges.index:
    e = np.where(edges.loc[var] == 1)
    ci_row = connection_indices[e, 0]
    ci_col = connection_indices[e, 1]
    s = strength.loc[var].values[e]
    observations[var] = pd.DataFrame(np.vstack((e,ci_row,ci_col,s)).T, columns=["2Dcol","3Drow","3Dcol","strn"])

In [25]:
observations['Group[T.Patient]']

,2Dcol,3Drow,3Dcol,strn
0,13.0,3.0,5.0,-0.675004
1,19.0,4.0,6.0,-0.255347
2,26.0,5.0,7.0,-0.089660
3,28.0,0.0,8.0,-0.948544
4,34.0,6.0,8.0,-1.432369
5,39.0,3.0,9.0,-0.721061
6,43.0,7.0,9.0,-0.849258
7,53.0,8.0,10.0,-2.039634
8,54.0,9.0,10.0,-0.935878
9,63.0,8.0,11.0,-0.709375


In [26]:
def get_observations(data, mod, thr_t, thr_p, alternative, ci):
    """

    :return:
    """
    lms = []
    for column in data.columns[3:]:
        lms.append(
            fit_linear_model(
                data,
                column,
                mod
            )
        )
    lms = pd.concat(lms, axis=1)

    df_tvals = lms[[col for col in lms.columns if 'tvals_' in col]]
    df_pvals = lms[[col for col in lms.columns if 'pvals_' in col]]
    qt_2 = scipy.stats.t.ppf(1 - thr_p / 2, data.shape[0] - lms.shape[0] - 1)
    qt = scipy.stats.t.ppf(1 - thr_p, data.shape[0] - lms.shape[0] - 1)
    
    if thr_t is not None:
        # calculate strength
        strength = (df_tvals.abs() - abs(thr_t)) * (1 - 2 * (df_tvals < 0))

        if alternative == "two.sided":
            edges = df_tvals.abs() > thr_t
        elif alternative == "less":
            edges = df_tvals < thr_t
        else:
            edges = df_tvals > thr_t
    else:
        if alternative == "two.sided":
            edges = df_pvals < thr_p
            strength = (df_tvals.abs() - qt_2)*(1-2*(df_tvals < 0))
        elif alternative == "less":
            edges = pd.DataFrame((df_pvals < 2*thr_p).values & (df_tvals < 0).values, columns=df_pvals.columns, index=df_pvals.index)
            strength = (df_tvals.abs() - qt)*(1-2*(df_tvals < 0))
        else:
            edges = pd.DataFrame((df_pvals < 2*thr_p).values & (df_tvals > 0).values, columns=df_pvals.columns, index=df_pvals.index)
            strength = (df_tvals.abs() - qt)*(1-2*(df_tvals < 0))
        edges

    observations = dict()
    for var in edges.index:
        e = np.where(edges.loc[var] == 1)
        ci_row = ci[e, 0]
        ci_col = ci[e, 1]
        s = strength.loc[var].values[e]
        observations[var] = np.vstack((e, ci_row, ci_col, s)).T.tolist()
    return 5

In [27]:
# pd.concat([data[['Group', 'Sex', 'Age']].sample(data.shape[0]).reset_index(drop=True), data.drop(columns=['Group', 'Sex', 'Age'])], axis=1).head()

In [28]:
%%time
r = get_observations(data, mod, None, 0.05, "less", connection_indices)

CPU times: user 1.28 s, sys: 6.96 ms, total: 1.29 s
Wall time: 1.29 s


In [29]:
def init_worker():
    signal.signal(signal.SIGINT, signal.SIG_IGN)

In [30]:
def apply_async(n_perm=1, n_core=1):
    st = time.time()
    pool = mp.Pool(n_core, init_worker)
    results_p = []
    try:
        for _ in range(n_perm):
            df_ = pd.concat([data[['Group', 'Sex', 'Age']].sample(data.shape[0]).reset_index(drop=True), data.drop(columns=['Group', 'Sex', 'Age'])], axis=1)
            results_p.append(pool.apply_async(get_observations, args=(df_, mod, None, 0.05, "less", connection_indices)))
    except KeyboardInterrupt:
        print('[Exception: Keyboard] Terminate MultiProcessing')
        pool.terminate()
        pool.join()
    except Exception as e:
        print('[Exception: Unknown] Terminate MultiProcessing')
        pool.terminate()
        pool.join()
    finally:
        # Close and wait for the pool to finish
        pool.close()
        pool.join()
        results_p = [r.get() for r in results_p]

    results = pd.concat(results_p, axis=1)
    et = time.time()
    print('[perfr-measurment] %d permutations with ncore=%d finished in %.2f [sec]' % (n_perm, n_core, et - st))

In [ ]:
if __name__ == '__main__':
    apply_async()

In [ ]:
if __name__ == '__main__':
    concurrency = 1
    n_perm = 1
    sema = Semaphore(concurrency)
    all_processes = []
    for i in range(n_perm):
        # once 20 processes are running, the following `acquire` call
        # will block the main process since `sema` has been reduced
        # to 0. This loop will continue only after one or more 
        # previously created processes complete.
        sema.acquire()
        p = Process(target=get_observations, args=(df_, mod, None, 0.05, "less", connection_indices, sema))
        all_processes.append(p)
        p.start()

    # inside main process, wait for all processes to finish
    for p in all_processes:
        p.join()

In [ ]:
pool = mp.Pool(1, init_worker)
df_ = pd.concat([data[['Group', 'Sex', 'Age']].sample(data.shape[0]).reset_index(drop=True), data.drop(columns=['Group', 'Sex', 'Age'])], axis=1)
tt = pool.apply_async(get_observations, args=(df_, mod, None, 0.05, "less", connection_indices))
tt.get()

#### Optimization of LM model

In [31]:
duration = []
n_iter = 100
for _ in range(n_iter):
    st = time.time()
    r1 = fit_linear_model(data, data.columns[3], mod)
    et = time.time()
    duration.append(et-st)
print(r1)
print('Average duration on %d iterations is: %.2f +- %.2f [ms]' % (n_iter, np.mean(duration) * 10**3, np.std(duration) * 10**3))

                  tvals_FAG_FAD  pvals_FAG_FAD
Group[T.Patient]       1.252376       0.217202
Sex[T.M]               0.520570       0.605337
Age                    0.933738       0.355653
Sex[T.M]:Age          -0.522405       0.604070
Average duration on 100 iterations is: 3.53 +- 1.54 [ms]


In [32]:
11.3 * 378 / 1000

4.271400000000001

In [33]:
mod

'Group + Sex * Age'

In [34]:
data_ = data.copy()
data_ = pd.concat([pd.get_dummies(data_[['Sex', 'Group']], drop_first=True), data_], axis=1)
data_.drop(columns=['Sex', 'Group'], inplace=True)
data_.head()
data_['Sex_m_Age'] = data_['Sex_M'] * data_['Age']

In [131]:
duration = []
n_iter = 100
for _ in range(n_iter):
    st = time.time()
    lm = smf.ols(
    formula='%s ~ %s' % (data.columns[3], 'Group_Patient + Sex_M + Age + Sex_m_Age'),
    data=data_
    )
    res = lm.fit()
    # return t-values and p-values for each independent variable
    r2 = pd.concat([res.tvalues, res.pvalues], axis=1).iloc[1:]
    r2.columns = ['tvals_%s' % data.columns[3], 'pvals_%s' % data.columns[3]]
    et = time.time()
    duration.append(et-st)
print(r2)
print('Average duration on %d iterations is: %.2f +- %.2f [ms]' % (n_iter, np.mean(duration) * 10**3, np.std(duration) * 10**3))

               tvals_FAG_FAD  pvals_FAG_FAD
Group_Patient       1.252376       0.217202
Sex_M               0.520570       0.605337
Age                 0.933738       0.355653
Sex_m_Age          -0.522405       0.604070
Average duration on 100 iterations is: 3.91 +- 1.64 [ms]


In [132]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                FAG_FAD   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                 -0.034
Method:                 Least Squares   F-statistic:                    0.6086
Date:                Wed, 28 Jul 2021   Prob (F-statistic):              0.659
Time:                        11:58:57   Log-Likelihood:                -17.714
No. Observations:                  48   AIC:                             45.43
Df Residuals:                      43   BIC:                             54.78
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.2536      0.416      0.610      0.545      -0.585       1.092
Group_Patient     0.1448      0.116      1.252      0.217      -0.088       0.378
Sex_M             0.2898      0.557      0.521      0.605      -0.833       1.413
Age               0.0276      0.030      0.934      0.356      -0.032       0.087
Sex_m_Age        -0.0208      0.040     -0.522      0.604      -0.101       0.060
==============================================================================
Omnibus:                       28.703   Durbin-Watson:                   2.098
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.814
Skew:                           1.138   Prob(JB):                     8.88e-35
Kurtosis:                      11.557   Cond. No.                         211.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [134]:
from sklearn import linear_model as lm
from scipy import stats

In [250]:
def fit_linear_model_sklearn(col_num=3):
    X=data_[['Group_Patient','Sex_M', 'Age', 'Sex_m_Age']]
    y=data_[data_.columns[col_num]]
    lr = lm.LinearRegression().fit(
        X,
        y
    )
    predictions = lr.predict(X)
    params = np.append(lr.intercept_,lr.coef_)
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))
    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(newX.shape[0]-newX.shape[1]))) for i in ts_b]

    return pd.DataFrame(np.vstack((ts_b[1:], p_values[1:])), columns=['Group_Patient','Sex_M', 'Age', 'Sex_m_Age'], 
                        index=['tvals_%s' % data_.columns[col_num], 'pvals_%s' % data_.columns[col_num]]).T

In [254]:
%%time
ncol = 6
rsk = fit_linear_model_sklearn(ncol)
rsm = fit_linear_model(data, data.columns[ncol], mod)

print(rsk)
print(rsm)

               tvals_FAG_F1D  pvals_FAG_F1D
Group_Patient       2.096530       0.041955
Sex_M               1.449636       0.154415
Age                -0.318207       0.751868
Sex_m_Age          -1.303951       0.199188
                  tvals_FAG_F1D  pvals_FAG_F1D
Group[T.Patient]       2.096530       0.041955
Sex[T.M]               1.449636       0.154415
Age                   -0.318207       0.751868
Sex[T.M]:Age          -1.303951       0.199188
CPU times: user 36.2 ms, sys: 32.2 ms, total: 68.4 ms
Wall time: 27.3 ms


#### Repeat for matrix of predictions

In [237]:
%%time
X=data_[['Group_Patient','Sex_M', 'Age', 'Sex_m_Age']]
y=data_[[col for col in data_.columns if not any(c in col for c in ['Group', 'Age', 'Sex'])]]
lr = lm.LinearRegression().fit(
    X,
    y
)

predictions = lr.predict(X)
params = np.vstack((lr.intercept_.T, lr.coef_.T)).T
newX = np.append(np.ones((len(X),1)), X, axis=1)
sq_error = np.sum((y-predictions)**2, axis=0)
MSE = sq_error/(newX.shape[0]-newX.shape[1])
inv_X = np.linalg.inv(np.dot(newX.T,newX)).diagonal()
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(newX.shape[0]-newX.shape[1]))) for i in ts_b]

pd.DataFrame(p_values).T.drop(index=0)

CPU times: user 71.6 ms, sys: 133 ms, total: 205 ms
Wall time: 53.3 ms


,0,1,2,3,4,5,6,7,8,9,...,368,369,370,371,372,373,374,375,376,377
1,0.217202,0.827539,0.409259,0.095369,0.053694,0.273836,0.257615,0.791474,0.207491,0.497917,...,0.558500,0.406136,0.606731,0.763239,0.691416,0.116725,0.078226,0.766097,0.529587,0.990189
2,0.605337,0.308433,0.747910,0.244855,0.982237,0.783752,0.754611,0.874261,0.268650,0.258656,...,0.436745,0.103964,0.284807,0.980776,0.727152,0.887343,0.750615,0.721868,0.609907,0.899781
3,0.355653,0.929768,0.725867,0.797013,0.218796,0.226650,0.515580,0.727207,0.631635,0.941888,...,0.739033,0.195296,0.574244,0.945784,0.933367,0.842713,0.973204,0.727502,0.735789,0.827159
4,0.604070,0.396148,0.698742,0.294805,0.949560,0.746340,0.750746,0.883090,0.355684,0.342207,...,0.590011,0.095179,0.268980,0.959946,0.732944,0.823091,0.746469,0.867462,0.570179,0.938074


In [262]:
def fit_linear_model_sklearn_matrix():
    X=data_[['Group_Patient','Sex_M', 'Age', 'Sex_m_Age']]
    y=data_[[col for col in data_.columns if not any(c in col for c in ['Group', 'Age', 'Sex'])]]
    lr = lm.LinearRegression().fit(
        X,
        y
    )
    predictions = lr.predict(X)
    params = np.vstack((lr.intercept_.T, lr.coef_.T)).T

    newX = np.append(np.ones((len(X),1)), X, axis=1)
    sq_error = np.sum((y-predictions)**2, axis=0)
    MSE = sq_error/(newX.shape[0]-newX.shape[1])
    inv_X = np.linalg.inv(np.dot(newX.T,newX)).diagonal()
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(newX.shape[0]-newX.shape[1]))) for i in ts_b]

    df = pd.DataFrame(p_values, columns=['Interception', 'Group_Patient','Sex_M', 'Age', 'Sex_m_Age'], index=['pvals_%s' % c for c in data_.columns[3:-1]]).T.iloc[1:]
    return df

In [263]:
df = fit_linear_model_sklearn_matrix()
df

,pvals_FAG_FAD,pvals_FAG_F1G,pvals_FAD_F1G,pvals_FAG_F1D,pvals_FAD_F1D,pvals_F1G_F1D,pvals_FAG_F1OG,pvals_FAD_F1OG,pvals_F1G_F1OG,pvals_F1D_F1OG,...,pvals_ORD_GRD,pvals_SMAG_GRD,pvals_SMAD_GRD,pvals_COBG_GRD,pvals_COBD_GRD,pvals_FMG_GRD,pvals_FMD_GRD,pvals_FMOG_GRD,pvals_FMOD_GRD,pvals_GRG_GRD
Group_Patient,0.131352,0.789066,0.311815,0.042168,0.019055,0.180506,0.166025,0.745446,0.123344,0.405824,...,0.472868,0.308605,0.527595,0.711491,0.626072,0.055763,0.032058,0.714920,0.440619,0.987951
Sex_M,0.525997,0.212373,0.693139,0.154853,0.978185,0.736142,0.701155,0.845912,0.175843,0.166945,...,0.340378,0.047513,0.190471,0.976392,0.668386,0.861883,0.696373,0.662105,0.531235,0.877087
Age,0.257821,0.913804,0.666858,0.752129,0.132679,0.139268,0.425158,0.668452,0.556263,0.928664,...,0.682539,0.113477,0.490611,0.933442,0.918215,0.807491,0.967094,0.668803,0.678671,0.788605
Sex_m_Age,0.524547,0.298392,0.634714,0.199667,0.938074,0.691263,0.696530,0.856688,0.257851,0.244665,...,0.508499,0.042052,0.176139,0.950818,0.675281,0.783673,0.691417,0.837620,0.486018,0.923986


In [265]:
lms[[c for c in lms.columns if 'pvals_' in c]]

,pvals_FAG_FAD,pvals_FAG_F1G,pvals_FAD_F1G,pvals_FAG_F1D,pvals_FAD_F1D,pvals_F1G_F1D,pvals_FAG_F1OG,pvals_FAD_F1OG,pvals_F1G_F1OG,pvals_F1D_F1OG,...,pvals_ORD_GRD,pvals_SMAG_GRD,pvals_SMAD_GRD,pvals_COBG_GRD,pvals_COBD_GRD,pvals_FMG_GRD,pvals_FMD_GRD,pvals_FMOG_GRD,pvals_FMOD_GRD,pvals_GRG_GRD
Group[T.Patient],0.217202,0.789066,0.321080,0.041955,0.013320,0.170953,0.118944,0.704606,0.133598,0.384832,...,0.252699,0.286681,0.519393,0.708470,0.629762,0.050199,0.067808,0.670287,0.462067,0.988430
Sex[T.M],0.605337,0.212373,0.698726,0.154415,0.976883,0.730352,0.664856,0.820447,0.187644,0.148759,...,0.130043,0.038408,0.181912,0.976132,0.671720,0.858495,0.741370,0.610521,0.550460,0.881941
Age,0.355653,0.913804,0.672862,0.751868,0.111639,0.130679,0.368346,0.617052,0.567015,0.925404,...,0.513255,0.097942,0.482009,0.932712,0.919087,0.802818,0.972174,0.618065,0.692836,0.796824
Sex[T.M]:Age,0.604070,0.298392,0.641208,0.199188,0.934384,0.684595,0.659743,0.832951,0.270837,0.224052,...,0.291746,0.033683,0.167814,0.950279,0.678554,0.778452,0.737083,0.811161,0.506457,0.927002


In [ ]:
lms.loc[]